In [1]:
import numpy as np

In [2]:
BATCH_SIZE = 6
IPU_BATCH = 16
GLOBAL_BATCH = BATCH_SIZE * IPU_BATCH

In [3]:
# num_atoms = np.abs(np.random.gamma(2, 20, size=GLOBAL_BATCH)).astype(int)
num_atoms = [17, 17, 24, 18, 20, 27, 26, 26, 67, 23, 33, 27, 22, 31, 20, 26, 33, 14, 29, 22, 30, 50, 25, 26, 27, 12, 23, 30, 19, 32, 19, 27, 33, 27, 30, 28, 33, 41, 21, 23, 24, 29, 17, 300, 23, 22, 31, 47, 22, 16, 33, 19, 17, 22, 22, 32, 38, 33, 30, 31, 27, 28, 34, 40, 26, 19, 17, 21, 18, 52, 26, 21, 25, 32, 29, 26, 43, 35, 30, 46, 31, 30, 28, 22, 43, 39, 32, 29, 18, 26, 33, 38, 26, 28, 49, 23]
num_atoms = np.asarray(num_atoms)
num_atoms

array([ 17,  17,  24,  18,  20,  27,  26,  26,  67,  23,  33,  27,  22,
        31,  20,  26,  33,  14,  29,  22,  30,  50,  25,  26,  27,  12,
        23,  30,  19,  32,  19,  27,  33,  27,  30,  28,  33,  41,  21,
        23,  24,  29,  17, 300,  23,  22,  31,  47,  22,  16,  33,  19,
        17,  22,  22,  32,  38,  33,  30,  31,  27,  28,  34,  40,  26,
        19,  17,  21,  18,  52,  26,  21,  25,  32,  29,  26,  43,  35,
        30,  46,  31,  30,  28,  22,  43,  39,  32,  29,  18,  26,  33,
        38,  26,  28,  49,  23])

In [4]:
class MolBatch:
    def __init__(self):
        self.num_atoms = 0
        self.num_mols = 0
        self.average_atom = 0
        self.indices = []

    def add_mol(self, num_atoms, idx):
        self.num_atoms += num_atoms
        self.num_mols += 1
        self.average_atom = self.num_atoms / self.num_mols
        self.indices.append(idx)
        return self

    def expected_sum(self, remaining_mean, batch_size):
        return self.num_atoms + ((batch_size - self.num_mols) * remaining_mean)

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(m: {self.num_mols},\ta: {self.num_atoms},\tav: {self.average_atom:.1f})"

In [7]:
def smart_packing(num_atoms):

    num_atoms = np.asarray(num_atoms)
    argsort_num_atoms = np.argsort(num_atoms)
    sorted_num_atoms = num_atoms[argsort_num_atoms]

    mol_batches = [MolBatch().add_mol(sorted_num_atoms[-ii-1], argsort_num_atoms[-ii-1]) for ii in range(IPU_BATCH)]
    sorted_num_atoms = sorted_num_atoms[:-IPU_BATCH]
    reverse_cumsum = np.sum(sorted_num_atoms) - np.cumsum(sorted_num_atoms) + sorted_num_atoms[-1]
    for ii, num_atom in enumerate(sorted_num_atoms):
        remaining_mean = reverse_cumsum[ii] / (len(sorted_num_atoms) - ii)
        idx_max_average = np.argmax([m.expected_sum(remaining_mean, BATCH_SIZE) * (m.num_mols < BATCH_SIZE) for m in mol_batches])
        mol_batches[idx_max_average].add_mol(num_atom, argsort_num_atoms[ii])
    
    packed_indices = [batch.indices for batch in mol_batches]

    # TODO: Remove below
    av_per_batch = [m.average_atom for m in mol_batches]
    print(np.min(av_per_batch), np.max(av_per_batch), np.mean(av_per_batch), "    ", np.min(num_atoms), np.max(num_atoms))
    mol_batches

    return packed_indices

packed_indices = smart_packing(num_atoms)

total_sum = 0
for pack in packed_indices:
    pack_num_atoms = []
    for idx in pack:
        pack_num_atoms.append(num_atoms[idx])
    total_sum += sum(pack_num_atoms)
    print(sum(pack_num_atoms), pack_num_atoms)
print(total_sum, np.sum(num_atoms))

28.833333333333332 62.666666666666664 31.0      12 300
376 [300, 12, 14, 16, 17, 17]
173 [67, 17, 17, 18, 23, 31]
173 [52, 17, 20, 25, 27, 32]
173 [50, 18, 21, 26, 26, 32]
173 [49, 18, 21, 26, 28, 31]
173 [47, 19, 22, 26, 28, 31]
173 [46, 19, 22, 26, 29, 31]
173 [43, 19, 23, 26, 29, 33]
173 [43, 19, 23, 26, 30, 32]
173 [41, 20, 23, 27, 30, 32]
174 [40, 21, 23, 27, 30, 33]
174 [39, 22, 24, 27, 29, 33]
174 [38, 22, 24, 27, 30, 33]
174 [38, 22, 25, 27, 29, 33]
174 [35, 22, 26, 28, 30, 33]
173 [34, 22, 26, 28, 30, 33]
2976 2976


In [6]:
argsort_num_atoms = np.argsort(num_atoms)
sorted_num_atoms = num_atoms[argsort_num_atoms]
sorted_num_atoms

mol_batches = [MolBatch().add_mol(sorted_num_atoms[-ii-1], argsort_num_atoms[-ii-1]) for ii in range(IPU_BATCH)]
for ii, num_atom in enumerate(sorted_num_atoms[:-IPU_BATCH]):
    idx = ii % IPU_BATCH
    mol_batches[idx].add_mol(num_atom, argsort_num_atoms[ii])

av_per_batch = [m.average_atom for m in mol_batches]
print(np.min(av_per_batch), np.max(av_per_batch), np.mean(av_per_batch), "    ", np.min(num_atoms), np.max(num_atoms))
mol_batches

27.833333333333332 68.83333333333333 31.0      12 300


[MolBatch(m: 6,	a: 413,	av: 68.8),
 MolBatch(m: 6,	a: 184,	av: 30.7),
 MolBatch(m: 6,	a: 172,	av: 28.7),
 MolBatch(m: 6,	a: 172,	av: 28.7),
 MolBatch(m: 6,	a: 173,	av: 28.8),
 MolBatch(m: 6,	a: 172,	av: 28.7),
 MolBatch(m: 6,	a: 171,	av: 28.5),
 MolBatch(m: 6,	a: 169,	av: 28.2),
 MolBatch(m: 6,	a: 170,	av: 28.3),
 MolBatch(m: 6,	a: 169,	av: 28.2),
 MolBatch(m: 6,	a: 168,	av: 28.0),
 MolBatch(m: 6,	a: 170,	av: 28.3),
 MolBatch(m: 6,	a: 169,	av: 28.2),
 MolBatch(m: 6,	a: 170,	av: 28.3),
 MolBatch(m: 6,	a: 167,	av: 27.8),
 MolBatch(m: 6,	a: 167,	av: 27.8)]